In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('./pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
# from backbone.trader_bot import TraderBot
random.seed(42)


In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:
tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Indices_group' in symbol.path]
tickers

['AUS200m',
 'DE30m',
 'FR40m',
 'HK50m',
 'JP225m',
 'STOXX50m',
 'UK100m',
 'US30m',
 'US500m',
 'USTECm',
 'IN50m']

In [5]:

tickers = [
    'AUS200m',
    'DE30m',
    'FR40m',
    'HK50m',
    'JP225m',
    'STOXX50m',
    'UK100m',
    'US30m',
    'US500m',
    'USTECm',
]


# tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Indices_group' in symbol.path and symbol.spread < 15]
print(tickers)


# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2014, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    # mt5.TIMEFRAME_M4,
    # mt5.TIMEFRAME_M5,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_M15,
    # mt5.TIMEFRAME_M30,
    # mt5.TIMEFRAME_M30,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['AUS200m', 'DE30m', 'FR40m', 'HK50m', 'JP225m', 'STOXX50m', 'UK100m', 'US30m', 'US500m', 'USTECm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AUS200m
AUS200m
AUS200m
DE30m
DE30m
DE30m
FR40m
FR40m
FR40m
HK50m
HK50m
HK50m
JP225m
JP225m
JP225m
STOXX50m
STOXX50m
STOXX50m
UK100m
UK100m
UK100m
US30m
US30m
US30m
US500m
US500m
US500m
USTECm
USTECm
USTECm


In [6]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{16385: Timestamp('2021-10-27 17:00:00'),
 16386: Timestamp('2021-10-27 17:00:00'),
 16388: Timestamp('2021-10-27 17:00:00')}

In [7]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00')
limit_date_train

Timestamp('2022-10-01 00:00:00')

In [8]:
class Bbands(Strategy):
    risk=3
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.85
    b_close_threshold = 0.5

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:

            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma:
                self.buy(size=self.risk / 100)
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma:
                self.sell(size=self.risk / 100)
            

In [9]:
class Bbands(Strategy):
    risk=3
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )

    def next(self):
        actual_close = self.data.Close[-1]
        
        if self.position:
            if self.position.is_long:
                if crossover(self.data.Close, self.middle_band) or actual_close <= self.sma:
                    self.position.close()

            if self.position.is_short:
                if crossover(self.middle_band, self.data.Close) or actual_close >= self.sma:
                    self.position.close()

        else:

            if crossover(self.data.Close, self.lower_band) and actual_close > self.sma:
                self.buy(size=self.risk / 100)
                
            if crossover(self.upper_band, self.data.Close) and actual_close < self.sma:
                self.sell(size=self.risk / 100)
            

In [10]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    Bbands, 
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    start_date = intervals_start_dates[interval]

    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        strategy,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

performance.drawdown_duration = pd.to_timedelta(performance.drawdown_duration)

performance.drawdown_duration = performance.drawdown_duration.dt.days



In [11]:

filter_performance = performance[
    (performance['return']>0) 
    # & (performance['return/dd']>=1) 
    # & (performance['stability_ratio'] > 0.8)
    # & (performance['interval'] <= 16385)
].sort_values(by=['drawdown_duration'], ascending=[True]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Bbands,US30m,16388,0.119218,3.581543,103581.543020,2.405630,90,54.545455,0.719350,11,0.506475,14.246387,103581.543020,337 days 08:00:00,1.488817,2.613261
0,Bbands,USTECm,16386,0.171536,3.678638,103678.637883,5.973981,107,52.380952,0.377813,21,0.223409,8.960177,103678.637883,337 days 10:00:00,0.615777,1.630464
0,Bbands,FR40m,16388,0.052916,1.632905,101632.904636,4.595695,117,46.666667,0.326124,15,0.125770,8.996540,101632.904636,337 days 08:00:00,0.355312,0.809081
0,Bbands,AUS200m,16386,0.423793,1.341340,101341.339782,2.271221,160,51.851852,0.394074,27,0.055336,8.523592,101341.339782,337 days 10:00:00,0.590581,1.366346
0,Bbands,HK50m,16388,0.320439,4.201386,104201.385830,6.375826,161,53.333333,0.588336,15,0.313196,9.710897,104201.385830,337 days 08:00:00,0.658956,1.579310
0,Bbands,US500m,16388,0.139157,3.679030,103679.030029,8.276177,168,50.000000,0.424171,14,0.308564,14.728149,103679.030029,337 days 08:00:00,0.444533,1.074041
0,Bbands,UK100m,16385,0.501559,1.348740,101348.739682,4.449536,171,62.000000,0.281588,50,0.030455,10.023725,101348.739682,338 days 02:00:00,0.303119,0.973112
0,Bbands,JP225m,16386,0.003668,1.092320,101092.319530,4.468544,202,62.500000,0.261871,16,0.064679,7.434236,101092.319530,337 days 10:00:00,0.244446,0.565923


In [12]:
for ticker, interval in zip(portfolio, intervals):
    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        Bbands,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )

    stats = bt_train.run()
    
    print(stats)

    bt_train.plot(filename=f'./{ticker}.html', resample=False)

Start                     2021-10-28 08:00:00
End                       2022-09-30 16:00:00
Duration                    337 days 08:00:00
Exposure Time [%]                   14.246387
Equity Final [$]                 103581.54302
Equity Peak [$]                  103860.94302
Return [%]                           3.581543
Buy & Hold Return [%]              -19.170174
Return (Ann.) [%]                    3.126928
Volatility (Ann.) [%]                4.346881
Sharpe Ratio                          0.71935
Sortino Ratio                        1.131248
Calmar Ratio                         1.299837
Max. Drawdown [%]                    -2.40563
Avg. Drawdown [%]                   -0.767899
Max. Drawdown Duration       90 days 00:00:00
Avg. Drawdown Duration       16 days 06:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                       7.434109
Worst Trade [%]                     -3.117417
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-10-28 08:00:00
End                       2022-09-30 18:00:00
Duration                    337 days 10:00:00
Exposure Time [%]                    8.960177
Equity Final [$]                103678.637883
Equity Peak [$]                 105218.147883
Return [%]                           3.678638
Buy & Hold Return [%]              -29.970282
Return (Ann.) [%]                    3.211508
Volatility (Ann.) [%]                8.500261
Sharpe Ratio                         0.377813
Sortino Ratio                        0.622471
Calmar Ratio                         0.537583
Max. Drawdown [%]                   -5.973981
Avg. Drawdown [%]                   -1.581197
Max. Drawdown Duration      107 days 02:00:00
Avg. Drawdown Duration       27 days 06:00:00
# Trades                                   21
Win Rate [%]                        52.380952
Best Trade [%]                       4.480631
Worst Trade [%]                     -4.516954
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-10-28 08:00:00
End                       2022-09-30 16:00:00
Duration                    337 days 08:00:00
Exposure Time [%]                     8.99654
Equity Final [$]                101632.904636
Equity Peak [$]                  102861.63899
Return [%]                           1.632905
Buy & Hold Return [%]              -16.193621
Return (Ann.) [%]                    1.442475
Volatility (Ann.) [%]                4.423092
Sharpe Ratio                         0.326124
Sortino Ratio                        0.474994
Calmar Ratio                         0.313875
Max. Drawdown [%]                   -4.595695
Avg. Drawdown [%]                   -1.151913
Max. Drawdown Duration      117 days 20:00:00
Avg. Drawdown Duration       31 days 11:00:00
# Trades                                   15
Win Rate [%]                        46.666667
Best Trade [%]                       2.535754
Worst Trade [%]                     -1.850015
Avg. Trade [%]                    

...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-10-27 17:00:00
End                       2022-09-30 19:00:00
Duration                    338 days 02:00:00
Exposure Time [%]                   10.023725
Equity Final [$]                101348.739682
Equity Peak [$]                 103206.658042
Return [%]                            1.34874
Buy & Hold Return [%]               -6.178303
Return (Ann.) [%]                    1.239778
Volatility (Ann.) [%]                4.402808
Sharpe Ratio                         0.281588
Sortino Ratio                        0.379254
Calmar Ratio                         0.278631
Max. Drawdown [%]                   -4.449536
Avg. Drawdown [%]                   -0.603603
Max. Drawdown Duration      171 days 09:00:00
Avg. Drawdown Duration       13 days 14:00:00
# Trades                                   50
Win Rate [%]                             62.0
Best Trade [%]                       1.520832
Worst Trade [%]                     -3.174749
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-10-28 08:00:00
End                       2022-09-30 18:00:00
Duration                    337 days 10:00:00
Exposure Time [%]                    7.434236
Equity Final [$]                 101092.31953
Equity Peak [$]                  103823.13616
Return [%]                            1.09232
Buy & Hold Return [%]                -9.93481
Return (Ann.) [%]                    1.019128
Volatility (Ann.) [%]                 3.89172
Sharpe Ratio                         0.261871
Sortino Ratio                        0.367287
Calmar Ratio                         0.228067
Max. Drawdown [%]                   -4.468544
Avg. Drawdown [%]                   -0.581057
Max. Drawdown Duration      202 days 22:00:00
Avg. Drawdown Duration       27 days 03:00:00
# Trades                                   16
Win Rate [%]                             62.5
Best Trade [%]                       1.549239
Worst Trade [%]                     -2.174095
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


In [13]:
from utils import walk_forward
from utils import plot_full_equity_curve
from utils import get_wfo_stats

def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    

for ticker, interval in zip(portfolio, intervals):
    
    print(ticker, interval)
    
    lookback_bars = 1000
    validation_bars = 250
    warmup_bars = 400

    params = {
        'bband_std' : list(np.arange(1, 2, 0.2)),
        'maximize': optim_func_2
    }

    stats = walk_forward(
        Bbands,
        symbols[ticker][interval], 
        lookback_bars=lookback_bars,
        validation_bars=validation_bars,
        warmup_bars=warmup_bars, 
        params=params,
        commission=7e-4, 
        margin=1/30, 
        cash=100_000
    )

    plot_full_equity_curve(
        symbols[ticker][interval], 
        stats, 
        warmup_bars=warmup_bars,
        lookback_bars=lookback_bars, 
        overlay_price=True
    )

    wfo_stats = get_wfo_stats(stats, warmup_bars, symbols[ticker][interval])

    for k, v in wfo_stats.items():
        print(k, v)

US30m 16388


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Start 2021-10-27 16:00:00
End 2024-08-30 18:00:00
Duration 1038 days 02:00:00
Exposure Time [%] 5.116783048398748
Equity Final [$] 96461.56909000025
Equity Peak [$] 103060.68412
Return [%] -5.7525946751604256
Buy & Hold Return [%] 35.18064603362259
Return (Ann.) [%] -1.9736173662408985
Volatility (Ann.) [%] 4.913638174067309
Sharpe Ratio 0.0
Sortino Ratio 0.0
Calmar Ratio 0.0
Max. Drawdown [%] -8.950227013105671
Avg. Drawdown [%] -5.313968170367539
Max. Drawdown Duration 865 days 20:00:00
Avg. Drawdown Duration 451 days 11:00:00
# Trades 129
Win Rate [%] 55.81395348837209
Best Trade [%] 1.8413705058397234
Worst Trade [%] -3.197246854069724
Avg. Trade [%] -0.0311472714467409
Max. Trade Duration 6 days 06:00:00
Avg. Trade Duration 1 days 06:00:00
Profit Factor 0.9356264316994206
Expectancy [%] -0.026036349004607984
SQN -0.4040102509867377
